In [1]:
import sys, os

if os.path.abspath('..') not in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

In [2]:
import torch
import torch.nn as nn
# from torchdiffeq import odeint_adjoint as odeint
from torchdiffeq import odeint
import altair as alt
import pandas as pd
import numpy as np

device = 'cuda:0' if torch.cuda.is_available() else None

In [3]:
class Lambda(nn.Module):
    def __init__(self):
        super().__init__()
        self.register_buffer('A', torch.tensor([[-0.1, 2.0], [-2.0, -0.1]]))
    def forward(self, t, y):
        dz = (y**3).mm(self.A)
        return dz #+ torch.randn_like(dz)

In [4]:
with torch.no_grad():
    true_y0 = torch.tensor([[2., 0.]]).to(device)
    t = torch.linspace(0., 25., 1000).to(device)
    true_y = odeint(Lambda().to(device), true_y0, t, method='dopri5')

In [5]:
alt.Chart(pd.DataFrame({
    'x': true_y[:, 0, 0].cpu().numpy(),
    'y': true_y[:, 0, 1].cpu().numpy(),
    't': t.cpu().numpy(),
})).mark_circle().encode(x='x:Q', y='y:Q', color=alt.Color('t:Q', scale=alt.Scale(scheme='plasma')))

alt.Chart(...)